In [1]:
# test different pre-trained models against the images used as "test" set during the training

#Importation des packages (il faut encore en supprimer quelques uns)
import numpy as np 
from keras.applications import  resnet50
from sklearn import metrics 
import os

import pandas as pd # Pour manipuler des DataFrames pandas

import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline

from keras.models import Sequential # Pour construire un réseau de neurones
from keras.layers import Dense, Conv2D # Pour instancier une couche dense
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Activation,GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.models import Model
import cv2
import keras
from numpy.random import seed
import tensorflow

from keras.preprocessing.image import ImageDataGenerator
import itertools # Pour créer des iterateurs

from sklearn import linear_model, preprocessing 



%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input

from sklearn import metrics 

Using TensorFlow backend.


In [2]:
#Conversion de l'image en array de format 28,28
def convert_image(X):
    X_img=[]
    for image in X:
        # Load image
        img=cv2.imread(image)
        # Resize image
        img=cv2.resize(img,(28,28))
        # for the black and white image
        if img.shape==(28, 28):
            img=img.reshape([28,28,1])
            img=np.concatenate([img,img,img],axis=2)
        # cv2 load the image BGR sequence color (not RGB)
        X_img.append(img[...,::-1])
    return np.array(X_img)

#Creation of dictionnary according to generator classes to convert the number labels (0,1,2,3,4,5) to tring labels ("autre","corbeau",....)
def metrique(arg_predict):
    dictionnaire=train_generator.class_indices
    dictionnaire_inv = {v: k for k, v in dictionnaire.items()}
    
    Keys=[]
    Values=[]

    for i in range(len(arg_predict)) :
        Keys.append(arg_predict[i])
        Values.append(dictionnaire_inv[arg_predict[i]])

    #print(metrics.classification_report(Y_test, Values))
    return Values

In [3]:
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_full.csv'
df=pd.read_csv(generateur_path)
df.head()

,path,labels,class,img_paths
0,./TGO 2019 - photos caméras/Parcelle 1 Guilbot...,incertain,oiseau,Rec_images/DSCF0029_incertain_0.JPG
1,./TGO 2019 - photos caméras/Parcelle 1 Guilbot...,ground,autre,Rec_images/DSCF0029_ground_1.JPG
2,./TGO 2019 - photos caméras/Parcelle 1 Guilbot...,corneille,corneille,Rec_images/DSCF0180_corneille_2.JPG
3,./TGO 2019 - photos caméras/Parcelle 1 Guilbot...,corneille,corneille,Rec_images/DSCF0180_corneille_3.JPG
4,./TGO 2019 - photos caméras/Parcelle 1 Guilbot...,pigeon,pigeon,Rec_images/DSCF0258_pigeon_4.JPG


In [4]:
liste_path=list(df["path"].unique())


In [5]:
#Paramètres
base_img_paths="/home/marcpozzo/Desktop/c3po/Images_aquises/"
#generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_bigger.csv'
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_full.csv'
#generateur_path='/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/Materiel/generateur.csv'
test_size=0.1
epochs=200
batch_size = 600
zoom_range = 1.25
horizontal_flip = True
Minimum_Number_Class=100
dropout_rate=0.1
#steps_per_epoch=len(data_train)//batch_size
steps_per_epoch=1
#validation_steps=len(data_test)//batch_size
validation_steps=1

#Importation du générateur contenant les chemins pour les imgettes grossies
#df=pd.read_csv("/home/marcpozzo/Desktop/c3po/Images_aquises/generateur.csv")
df=pd.read_csv(generateur_path)
df.drop('labels',inplace=True,axis=1)
df.shape


#Récupération des labels à partir du fichier
df["class"].unique()
for c in df:
    print(''+c+':',len(df[c].unique()))
df["class"].unique()

All_Unique=df["class"].unique()
Utilisable=[]
for i in df["class"].unique():
    if df["class"][df["class"]==i].count()>Minimum_Number_Class:
        Utilisable.append(i)
Utilisable
Non_Utilisable=set(All_Unique)-set(Utilisable)
Non_Utilisable
for i in Non_Utilisable:
    df=df[df["class"]!=i]
df=df[df["class"]!="oiseau"]  
df["class"].unique()


for i in range(len(df["class"])):
    image_name=df["img_paths"].iloc[i]
    df["img_paths"].iloc[i]=os.path.join(base_img_paths,image_name)
    


seed(1)
tensorflow.random.set_seed(2)

path: 14
class: 12
img_paths: 6220


In [6]:
#for i in range(len(liste_path)):
j=0
for i in liste_path:
    print(liste_path[j],len(df[df["path"]==i]))
    base=df[df["path"]==i]


./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 34
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 34
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 35
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 6
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 74
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 43
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 152
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 76
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 228
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 985
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 1117
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 241
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 2396
./TGO 2019 - photos caméras/Parcelle 1 Guilbot/DCIM/101MEDIA 546


In [7]:
#data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 10,
        zoom_range = zoom_range,
        horizontal_flip = horizontal_flip
        )






train_generator = train_data_generator.flow_from_dataframe(dataframe=df,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = batch_size)


Found 5967 validated image filenames belonging to 6 classes.


'\ntest_data_generator = ImageDataGenerator(\n    preprocessing_function = preprocess_input)\n\ntest_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,\n                                                          directory="",\n                                                           x_col = "img_paths",\n                                                           class_mode ="sparse",\n                                                          target_size = (28 , 28), \n                                                          batch_size = batch_size)\n'

In [22]:
generator=ImageDataGenerator(preprocessing_function = preprocess_input)


j=0
for i in liste_path[-5:len(liste_path)]:
    #print(liste_path[j],len(df[df["path"]==i]))
    print(i)
    base=df[df["path"]==i]


    data_generator =generator.flow_from_dataframe(dataframe=base,
                                                              directory="",
                                                               x_col = "img_paths",
                                                               class_mode ="sparse",
                                                              target_size = (28 , 28), 
                                                              batch_size = batch_size)
    data_img=convert_image(base.img_paths)
    #Testing all the neurone modes and display the principal metrics and crosstables.
    ##The best score is 6c_rob trained with original dataset. 
    folder="zoom_models/"
    #Neurones=['6c_rob','drop_out.50','z1.1','zoom_0.9:1.3_flip','zoom_1.3']
    Y_test=base["class"]
    n='6c_rob'

    model = load_model(folder+n)
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    features = intermediate_layer_model.predict(preprocess_input(data_img))
    arg_predict=features.argmax(axis=1)
    pred=metrique(arg_predict)
    print("")
    print("Mesures de qualité du modèle: ",i)
    print("")
    print(n,metrics.classification_report(Y_test,pred))
    print("")
    print("table de cotingeance pour le modèle: ",i)
    print("")
    print(pd.crosstab(Y_test,np.array(pred)))
    

./DonneesPI/timeLapsePhotos_Pi1_4
Found 985 validated image filenames belonging to 6 classes.

Mesures de qualité du modèle:  ./DonneesPI/timeLapsePhotos_Pi1_4

6c_rob               precision    recall  f1-score   support

       autre       0.96      0.95      0.95       510
   chevreuil       0.96      0.88      0.92        75
   corneille       0.93      0.96      0.95       274
      faisan       0.87      0.91      0.89        67
       lapin       0.90      0.86      0.88        50
      pigeon       0.70      0.78      0.74         9

    accuracy                           0.94       985
   macro avg       0.89      0.89      0.89       985
weighted avg       0.94      0.94      0.94       985


table de cotingeance pour le modèle:  ./DonneesPI/timeLapsePhotos_Pi1_4

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        484          3          9       6      5       3
chevreuil      8         66  

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



Mesures de qualité du modèle:  ./DonneesPI/timeLapsePhotos_Pi1_2

6c_rob               precision    recall  f1-score   support

       autre       0.95      0.95      0.95       132
   chevreuil       0.98      0.95      0.96        58
   corneille       0.96      0.92      0.94        51
      faisan       0.00      0.00      0.00         0
       lapin       0.00      0.00      0.00         0
      pigeon       0.00      0.00      0.00         0

    accuracy                           0.95       241
   macro avg       0.48      0.47      0.48       241
weighted avg       0.96      0.95      0.95       241


table de cotingeance pour le modèle:  ./DonneesPI/timeLapsePhotos_Pi1_2

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        126          1          2       1      1       1
chevreuil      3         55          0       0      0       0
corneille      3          0         47       1      0       0

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



Mesures de qualité du modèle:  ./DonneesPI/timeLapsePhotos_Pi1_1

6c_rob               precision    recall  f1-score   support

       autre       0.99      0.99      0.99      1540
   corneille       0.97      0.96      0.97       285
      faisan       0.97      0.99      0.98       138
       lapin       0.68      0.79      0.73        19
      pigeon       0.98      0.96      0.97       414

    accuracy                           0.98      2396
   macro avg       0.92      0.94      0.93      2396
weighted avg       0.98      0.98      0.98      2396


table de cotingeance pour le modèle:  ./DonneesPI/timeLapsePhotos_Pi1_1

col_0      autre  corneille  faisan  lapin  pigeon
class                                             
autre       1531          3       1      0       5
corneille      1        274       1      7       2
faisan         2          0     136      0       0
lapin          1          2       0     15       1
pigeon        11          3       2      0     398
./Donn

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [9]:
#target_img=base["class"]

['./DonneesPI/timeLapsePhotos_Pi1_4',
 './DonneesPI/timeLapsePhotos_Pi1_3',
 './DonneesPI/timeLapsePhotos_Pi1_2',
 './DonneesPI/timeLapsePhotos_Pi1_1',
 './DonneesPI/timeLapsePhotos_Pi1_0']


Mesures de qualité du modèle:  6c_rob

6c_rob               precision    recall  f1-score   support

       autre       0.98      0.98      0.98       274
   corneille       0.98      0.99      0.98        88
      faisan       0.00      0.00      0.00         0
      pigeon       0.98      0.97      0.98       184

    accuracy                           0.98       546
   macro avg       0.74      0.74      0.74       546
weighted avg       0.98      0.98      0.98       546


table de cotingeance pour le modeèle:  6c_rob

col_0      autre  corneille  faisan  pigeon
class                                      
autre        269          1       1       3
corneille      1         87       0       0
pigeon         4          1       0     179


/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [5]:
#Testing all the neurone modes and display the principal metrics and crosstables.
##The best score is 6c_rob trained with original dataset. 

folde='iteration_models/'
Neurones=['iteration_no_transformation','z1.2_r_20','z1.2_r_20_drpt_0.1','z1.2_r_20_hflip_rotation_10']
for n in Neurones:
    model = load_model(folde+n)
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    X_test_features = intermediate_layer_model.predict(preprocess_input(X_test_img))
    arg_predict=X_test_features.argmax(axis=1)


    pred=metrique(arg_predict)
    print("")
    print("Mesures de qualité du modèle: ", n)
    print("")
    print(n,metrics.classification_report(Y_test,pred))
    print("")
    print("table de cotingeance pour le modeèle: ", n)
    print("")
    print(pd.crosstab(Y_test,np.array(pred)))


Mesures de qualité du modèle:  iteration_no_transformation

iteration_no_transformation               precision    recall  f1-score   support

       autre       0.80      0.96      0.88       342
   chevreuil       0.67      0.43      0.52        14
   corneille       0.92      0.71      0.80       107
      faisan       0.50      0.55      0.52        33
       lapin       0.50      0.11      0.18        18
      pigeon       0.85      0.54      0.66        83

    accuracy                           0.80       597
   macro avg       0.71      0.55      0.59       597
weighted avg       0.80      0.80      0.78       597


table de cotingeance pour le modeèle:  iteration_no_transformation

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        330          1          2       7      0       2
chevreuil      8          6          0       0      0       0
corneille     21          1         76       5     

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Mesures de qualité du modèle:  z1.2_r_20_hflip_rotation_10

z1.2_r_20_hflip_rotation_10               precision    recall  f1-score   support

       autre       0.81      0.94      0.87       342
   chevreuil       0.73      0.57      0.64        14
   corneille       0.69      0.78      0.73       107
      faisan       0.67      0.42      0.52        33
       lapin       0.00      0.00      0.00        18
      pigeon       0.93      0.52      0.67        83

    accuracy                           0.79       597
   macro avg       0.64      0.54      0.57       597
weighted avg       0.77      0.79      0.77       597


table de cotingeance pour le modeèle:  z1.2_r_20_hflip_rotation_10

col_0      autre  chevreuil  corneille  faisan  lapin  pigeon
class                                                        
autre        323          3         11       4      0       1
chevreuil      5          8          1       0      0       0
corneille     19          0         83       2     